In [3]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import calendar
from scipy.stats import chi2_contingency

from sklearn import tree
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import sys
import os
import sys
import os

os.chdir(sys.path[3])

In [4]:
raw_data = pd.read_csv(r'data/process/new_data.csv', skiprows=[1, 2]).reset_index()
raw_data

,index,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,Q33_9_TEXT,Q34,Q35,Q36,Q37,Q38,Q39,Q39_10_TEXT,Q40,Q40_3_TEXT
0,0,4/21/2022 14:05,4/21/2022 14:10,IP Address,169.231.161.135,100,318,True,4/21/2022 14:10,R_sRQ635z5e2hc1t7,...,NaN,1996.0,Woman,"$200,000 or more per year",One,"Graduate degree (e.g., Master’s, Professional ...","European (e.g., British Isles, German, French,...",NaN,Email,NaN
1,1,4/14/2022 17:28,4/14/2022 17:41,IP Address,128.111.106.41,22,770,False,4/21/2022 17:41,R_21uo7CnrNd8VNIx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,4/21/2022 19:27,4/21/2022 19:31,IP Address,70.77.83.154,100,229,True,4/21/2022 19:31,R_7V5ufQv2WtbkiSR,...,NaN,1973.0,Man,I prefer to not answer,Three,Bachelor’s degree (university 4-year degree) ...,"European (e.g., British Isles, German, French,...",NaN,"A bicycle registry/recovery system (e.g., Bike...",NaN
3,3,4/14/2022 19:04,4/14/2022 20:18,IP Address,70.185.131.128,22,4454,False,4/21/2022 20:18,R_qPdUVtdN6wMTtIt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4/22/2022 9:08,4/22/2022 9:24,IP Address,24.52.45.71,100,968,True,4/22/2022 9:24,R_3F2LBFhUg7Q8AbI,...,NaN,1959.0,Man,"Between $100,000 and $149,999 per year",Three,Associate’s/vocational/technical degree (2- or...,Multi-ethnic (2+ ethnic groups),NaN,Email,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,2020,6/6/2022 22:07,6/6/2022 22:14,IP Address,47.146.75.18,100,423,True,6/6/2022 22:14,R_3fYRblhWSYPWCj8,...,NaN,1970.0,Woman,"Between $100,000 and $149,999 per year",Three,Bachelor’s degree (university 4-year degree) ...,"European (e.g., British Isles, German, French,...",NaN,Email,NaN
2021,2021,5/24/2022 12:17,5/24/2022 12:20,IP Address,174.90.223.242,69,191,False,6/7/2022 12:20,R_3O7a8xS7l3i8EZG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,2022,6/8/2022 3:19,6/8/2022 3:30,IP Address,108.211.181.111,100,697,True,6/8/2022 3:30,R_3gO85Ni57rH0pTr,...,NaN,NaN,Prefer not to say,I prefer to not answer,I prefer to not answer,"Graduate degree (e.g., Master’s, Professional ...",I prefer to not answer,NaN,Other (please specify):,acquientance
2023,2023,5/25/2022 13:39,5/25/2022 13:39,IP Address,50.205.153.166,2,12,False,6/8/2022 13:39,R_1jwTkRmNyNow9Tz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
raw_data.groupby('Q35').count()

,index,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,Q33,Q33_9_TEXT,Q34,Q36,Q37,Q38,Q39,Q39_10_TEXT,Q40,Q40_3_TEXT
Q35,,,,,,,,,,,,,,,,,,,,,
Man,1060,1060,1060,1060,1060,1060,1060,1060,1060,1060,...,64,13,1039,1057,1060,1058,1051,28,1051,90
Non-binary / third gender,21,21,21,21,21,21,21,21,21,21,...,4,1,20,21,21,21,21,1,20,2
Prefer not to say,35,35,35,35,35,35,35,35,35,35,...,4,1,26,35,35,35,34,3,35,3
Woman,614,614,614,614,614,614,614,614,614,614,...,34,11,599,613,614,614,612,18,611,52


In [7]:
raw_data['age'] = 2022 - raw_data['Q34']
raw_data['age']

0       26.0
1        NaN
2       49.0
3        NaN
4       63.0
        ... 
2020    52.0
2021     NaN
2022     NaN
2023     NaN
2024    59.0
Name: age, Length: 2025, dtype: float64

In [11]:
raw_data['age'].notna()
res = raw_data[raw_data['age'].notna()]['age']

In [20]:
res = res.reset_index()


ValueError: cannot insert level_0, already exists

In [21]:
leny =len(res)
leny

1684

In [23]:
res[20>res['age']>30]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [29]:
for i in range(10):
    print('{} - {}: {}'.format(10*i,10* (i+1),res[res['age'].between(10*i,10* (i+1))].count()/leny))

0 - 10: level_0    0.001188
index      0.001188
age        0.001188
dtype: float64
10 - 20: level_0    0.029097
index      0.029097
age        0.029097
dtype: float64
20 - 30: level_0    0.187055
index      0.187055
age        0.187055
dtype: float64
30 - 40: level_0    0.364608
index      0.364608
age        0.364608
dtype: float64
40 - 50: level_0    0.232779
index      0.232779
age        0.232779
dtype: float64
50 - 60: level_0    0.165677
index      0.165677
age        0.165677
dtype: float64
60 - 70: level_0    0.08848
index      0.08848
age        0.08848
dtype: float64
70 - 80: level_0    0.03266
index      0.03266
age        0.03266
dtype: float64
80 - 90: level_0    0.001188
index      0.001188
age        0.001188
dtype: float64
90 - 100: level_0    0.0
index      0.0
age        0.0
dtype: float64


In [ ]:
res['age'].between(10,20))].count()